In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Styling
plt.style.use('tdrstyle.mplstyle')

In [2]:
from rootpy.io import root_open
import ROOT
ROOT.gROOT.SetBatch(True)

Welcome to JupyROOT 6.10/09


In [3]:
infile_r = None  # input file handle

def load_pgun():
  global infile_r
  infile = '../test7/ntuple_SingleMuon_Toy_2GeV_add.6.root'
  #if use_condor:
  #  infile = 'root://cmsio5.rc.ufl.edu//store/user/jiafulow/L1MuonTrigger/P2_10_1_5/SingleMuon_Toy_2GeV/'+infile
  infile_r = root_open(infile)
  tree = infile_r.ntupler.tree
  #tree = TreeChain('ntupler/tree', [infile])
  print('[INFO] Opening file: %s' % infile)

  # Define collection
  tree.define_collection(name='hits', prefix='vh_', size='vh_size')
  tree.define_collection(name='tracks', prefix='vt_', size='vt_size')
  tree.define_collection(name='particles', prefix='vp_', size='vp_size')
  return tree

In [75]:
maxEvents = 100
#maxEvents = 200000

kDT, kCSC, kRPC, kGEM, kME0 = 0, 1, 2, 3, 4

eta_bins = (1.2, 1.4, 1.55, 1.7, 1.8, 1.98, 2.15, 2.5)
eta_bins = eta_bins[::-1]
pt_bins = (-0.50, -0.333333, -0.25, -0.20, -0.15, -0.10, -0.05, 0.05, 0.10, 0.15, 0.20, 0.25, 0.333333, 0.50)
nlayers = 12  # 5 (CSC) + 4 (RPC) + 3 (GEM)

def find_eta_bin(eta):
  ieta = np.digitize((abs(eta),), eta_bins[1:])[0]  # skip lowest edge
  ieta = np.clip(ieta, 0, len(eta_bins)-2)
  return ieta

def find_endsec(endcap, sector):
  endsec = (sector - 1) if endcap == 1 else (sector - 1 + 6)
  return endsec

def delta_phi(lhs, rhs):  # in radians
  rad = lhs - rhs
  while rad <  -np.pi:  rad += np.pi*2
  while rad >= +np.pi:  rad -= np.pi*2
  return rad

# Make EMTF image
# A m-by-n matrix has m rows and n columns
class EMTFImage(object):
  def __init__(self, superstrip_size=16):
    self.superstrip_size = superstrip_size
    
    l_lut = np.zeros((5,5,5), dtype=np.int32) - 99  # (type, station, ring) -> layer
    l_lut[1,1,4] = 0  # ME1/1a
    l_lut[1,1,1] = 0  # ME1/1b
    l_lut[1,1,2] = 1  # ME1/2
    l_lut[1,1,3] = 1  # ME1/3
    l_lut[1,2,1] = 2  # ME2/1
    l_lut[1,2,2] = 2  # ME2/2
    l_lut[1,3,1] = 3  # ME3/1
    l_lut[1,3,2] = 3  # ME3/2
    l_lut[1,4,1] = 4  # ME4/1
    l_lut[1,4,2] = 4  # ME4/2
    l_lut[2,1,2] = 1  # RE1/2
    l_lut[2,2,2] = 2  # RE2/2
    l_lut[2,3,1] = 3  # RE3/1
    l_lut[2,3,2] = 3  # RE3/2
    l_lut[2,3,3] = 3  # RE3/3
    l_lut[2,4,1] = 4  # RE4/1
    l_lut[2,4,2] = 4  # RE4/2
    l_lut[2,4,3] = 4  # RE4/3
    l_lut[3,1,1] = 0  # GE1/1
    l_lut[3,2,1] = 2  # GE2/1
    l_lut[4,1,1] = 0  # ME0
    self.l_lut = l_lut
    assert(self.l_lut.max() < nlayers)
    
    m_lut = np.zeros((5,5,5,2), dtype=np.int32) - 99  # (type, station, ring, fr) -> row
    m_lut[1,1,4,0] = 4  # ME1/1a (R)
    m_lut[1,1,4,1] = 3  # ME1/1a (F)
    m_lut[1,1,1,0] = 4  # ME1/1b (R)
    m_lut[1,1,1,1] = 3  # ME1/1b (F)
    m_lut[1,1,2,0] = 1  # ME1/2 (R)
    m_lut[1,1,2,1] = 0  # ME1/2 (F)
    m_lut[1,1,3,:] = 0  # ME1/3
    m_lut[1,2,1,:] = 6  # ME2/1
    m_lut[1,2,2,:] = 6  # ME2/2
    m_lut[1,3,1,:] = 7  # ME3/1
    m_lut[1,3,2,:] = 7  # ME3/2
    m_lut[1,4,1,:] = 9  # ME4/1
    m_lut[1,4,2,:] = 9  # ME4/2
    m_lut[2,1,2,:] = 2  # RE1/2
    m_lut[2,2,2,:] = 5  # RE2/2
    m_lut[2,3,1,:] = 8  # RE3/1
    m_lut[2,3,2,:] = 8  # RE3/2
    m_lut[2,3,3,:] = 8  # RE3/3
    m_lut[2,4,1,:] = 10 # RE4/1
    m_lut[2,4,2,:] = 10 # RE4/2
    m_lut[2,4,3,:] = 10 # RE4/3
    m_lut[3,1,1,:] = 2  # GE1/1
    m_lut[3,2,1,:] = 5  # GE2/1
    m_lut[4,1,1,:] = 0  # ME0
    self.m_lut = m_lut
    self.m_size = 11
    assert(self.m_lut.max() < self.m_size)
    
    self.n_size = 5040 // self.superstrip_size
    
    t_lut = np.zeros((11,7,2), dtype=np.int32) - 99  # (row,zone) -> (min theta, max theta)
    t_lut[0,0] = 4,17   # ME0
    t_lut[0,1] = 17,23  # ME0
    t_lut[1,0] = 4,17   # ME0
    t_lut[1,1] = 17,23  # ME0
    t_lut[2,1] = 17,26  # GE1/1
    t_lut[2,2] = 24,37  # GE1/1
    t_lut[2,3] = 35,45  # GE1/1
    t_lut[2,4] = 40,52  # GE1/1
    t_lut[3,0] = 4,17   # ME1/1 (F)
    t_lut[3,1] = 16,26  # ME1/1 (F)
    t_lut[3,2] = 24,37  # ME1/1 (F)
    t_lut[3,3] = 34,43  # ME1/1 (F)
    t_lut[3,4] = 41,53  # ME1/1 (F)
    t_lut[4,0] = 4,17   # ME1/1 (R)
    t_lut[4,1] = 16,26  # ME1/1 (R)
    t_lut[4,2] = 24,37  # ME1/1 (R)
    t_lut[4,3] = 34,43  # ME1/1 (R)
    t_lut[4,4] = 41,53  # ME1/1 (R)
    #
    t_lut[0,4] = 46,54  # ME1/2 (F)
    t_lut[0,5] = 52,67  # ME1/2 (F)
    t_lut[0,6] = 65,87  # ME1/2 (F)
    t_lut[1,4] = 46,54  # ME1/2 (R)
    t_lut[1,5] = 52,67  # ME1/2 (R)
    t_lut[1,6] = 65,87  # ME1/2 (R)
    t_lut[2,5] = 52,72  # RE1/2
    t_lut[2,6] = 68,84  # RE1/2
    #
    t_lut[5,0] = 7,19   # GE2/1
    t_lut[5,1] = 18,24  # GE2/1
    t_lut[5,2] = 23,35  # GE2/1
    t_lut[5,3] = 34,45  # GE2/1
    t_lut[5,4] = 40,46  # GE2/1
    t_lut[5,5] = 56,68  # RE2/2+3
    t_lut[5,6] = 64,76  # RE2/2+3
    t_lut[6,0] = 4,17   # ME2/1
    t_lut[6,1] = 16,25  # ME2/1
    t_lut[6,2] = 24,36  # ME2/1
    t_lut[6,3] = 34,43  # ME2/1
    t_lut[6,4] = 40,49  # ME2/1
    t_lut[6,5] = 52,67  # ME2/2
    t_lut[6,6] = 65,87  # ME2/2
    #
    t_lut[7,0] = 4,17   # ME3/1
    t_lut[7,1] = 16,25  # ME3/1
    t_lut[7,2] = 24,36  # ME3/1
    t_lut[7,3] = 34,40  # ME3/1
    t_lut[7,4] = 44,54  # ME3/2
    t_lut[7,5] = 52,67  # ME3/2
    t_lut[7,6] = 64,87  # ME3/2
    t_lut[8,0] = 4,20   # RE3/1
    t_lut[8,1] = 20,24  # RE3/1
    t_lut[8,2] = 24,32  # RE3/1
    t_lut[8,3] = 40,40  # RE3/2+3
    t_lut[8,4] = 40,52  # RE3/2+3
    t_lut[8,5] = 48,72  # RE3/2+3
    t_lut[8,6] = 60,84  # RE3/2+3
    #
    t_lut[9,0] = 4,17   # ME4/1
    t_lut[9,1] = 16,25  # ME4/1
    t_lut[9,2] = 24,35  # ME4/1
    t_lut[9,3] = 38,43  # ME4/2
    t_lut[9,4] = 41,54  # ME4/2
    t_lut[9,5] = 52,67  # ME4/2
    t_lut[9,6] = 64,87  # ME4/2
    t_lut[10,0] = 8,16  # RE4/1
    t_lut[10,1] = 16,28 # RE4/1
    t_lut[10,2] = 24,28 # RE4/1
    t_lut[10,3] = 36,44 # RE4/2+3
    t_lut[10,4] = 44,52 # RE4/2+3
    t_lut[10,5] = 52,64 # RE4/2+3
    t_lut[10,6] = 64,84 # RE4/2+3
    self.t_lut = t_lut
    
    # ME1/1, ME1/2, ME2, ME3, ME4, RE1
    # RE2, RE3, RE4, GE1/1, GE2/1, ME0
    s_bend_lut =[-0.059926, -0.065415, -0.149701,  0.086035,  0.108797,  1.000000,
                  1.000000,  1.000000,  1.000000, -0.515074, -0.599277, -0.073872,]
    self.s_bend_lut = np.array(s_bend_lut, dtype=np.float32)

  def get_layer(self, hit):
    index = (hit.type, hit.station, hit.ring)
    lay = self.l_lut[index]
    return lay
  
  def get_row(self, hit):
    index = (hit.type, hit.station, hit.ring, hit.fr)
    m = self.m_lut[index]
    return m

  def get_col(self, hit):
    hit_phi = np.asarray(hit.emtf_phi, dtype=np.int32)
    n = hit_phi // self.superstrip_size
    return n
  
  def get_zones(self, hit):
    m = self.get_row(hit)
    t = self.t_lut[m]
    hit_theta = np.asarray(hit.emtf_theta, dtype=np.int32)
    zones = np.logical_and(t[:,0] <= hit_theta, hit_theta <= t[:,1])
    zones = np.where(zones)[0]
    return zones

  def get_chn_bend(self, hit):
    if hit.type == kCSC:
      bend = hit.bend
      bend *= hit.endcap
    elif hit.type == kGEM:
      bend = hit.bend
      bend *= hit.endcap
    else:
      bend = hit.bend
    lay = self.get_layer(hit)
    s = self.s_bend_lut[lay]
    bend *= s
    return bend
  
  def get_chn_theta(self, hit):
    theta = (hit.emtf_theta - 3.)/83.
    return theta
  
  def get_chn_mphi(self, hit):
    n = self.get_col(hit)
    mphi = hit.emtf_phi - (n * self.superstrip_size)
    return mphi
  
  def get_chn(self, hit):
    chn = (self.get_chn_bend(hit),
           self.get_chn_theta(hit),
           self.get_chn_mphi(hit))
    chn = np.asarray(chn, dtype=np.float32)
    return chn
  
  def is_intime(self, hit):
    if hit.type == kCSC:
      return hit.bx == -1 or hit.bx == 0
    else:
      return hit.bx == 0
  
  def __call__(self, hits):
    amap = {}
    for ihit, hit in enumerate(hits):
      if self.is_intime(hit):
        m = self.get_row(hit)
        n = self.get_col(hit)
        zones = self.get_zones(hit)
        for z in zones:
          m += (z * self.m_size)
          amap.setdefault((m,n), []).append(hit)
      
    image_pixels = np.zeros((50,2), dtype=np.int32) - 99
    image_channels = np.zeros((50,3), dtype=np.float32) + np.nan
    
    i = 0
    for k, v in amap.iteritems():
      (m,n) = k
      hits_mn = v
      h = hits_mn[0]  # arbitrarily pick the first hit
      chn = self.get_chn(h)
      
      image_pixels[i] = (m,n)
      image_channels[i] = chn
      i += 1
      
    return image_pixels, image_channels

make_emtf_image = EMTFImage()


# Assign EMTF label
class EMTFLabel(object):
  def __init__(self):
    self.pt_bins = np.linspace(-0.5, 0.5, num=20)
    self.eta_bins = eta_bins
    self.phi_bins = np.linspace(-40, 40, num=80)
  
  def find_pt_bin(self, part):
    pt = np.true_divide(part.q, part.pt)
    ipt = np.digitize((pt,), self.pt_bins[1:])[0]  # skip lowest edge
    ipt = np.clip(ipt, 0, len(self.pt_bins)-2)
    return ipt
  
  def find_sector_phi(self, glob, sector):  # glob in deg, sector in [1,6]
    # sector 1 starts at 15 deg
    loc = glob - 15. - (60. * (sector-1))
    # but here I'm assuming sector 1 centers at 40 deg
    center = 40.
    dphi = delta_phi(loc, center)
    return dphi
  
  def find_phi_bin(self, part, sector):
    phi = self.find_sector_phi(np.rad2deg(part.phi), (sector%6)+1)
    iphi = np.digitize((phi,), self.phi_bins[1:])[0]  # skip lowest edge
    iphi = np.clip(iphi, 0, len(self.phi_bins)-2)
    return iphi
  
  def find_eta_bin(self, part):
    eta = abs(part.eta)
    ieta = np.digitize((eta,), self.eta_bins[1:])[0]  # skip lowest edge
    ieta = np.clip(ieta, 0, len(self.eta_bins)-2)
    return ieta
  
  def __call__(self, part, sector):
    label = (self.find_pt_bin(part),
             self.find_phi_bin(part, sector),
             self.find_eta_bin(part))
    label = np.asarray(label, dtype=np.int32)
    return label
  
assign_emtf_label = EMTFLabel()

In [76]:
#FIXME: format 1/pT
#FIXME: dont reduce strip_unit by 8
def process(verbose=1):
  tree = load_pgun()
  
  out_image_pixels = []
  out_image_channels = []
  out_labels = []

  # Loop over events
  for ievt, evt in enumerate(tree):
    if maxEvents != -1 and ievt == maxEvents:
      break
    
    if verbose and (ievt % 1000 == 0):  print("Processing event: {0}".format(ievt))
    
    # Skip events with very few hits
    if not len(evt.hits) >= 4:
      continue
    
    # Skip events without ME1 hits
    has_ME1 = False
    for ihit, hit in enumerate(evt.hits):
      if hit.type == kCSC and hit.station == 1:
        has_ME1 = True
        break
      elif hit.type == kME0 and hit.station == 1:
        has_ME1 = True
        break
    if not has_ME1:
      continue
    
    part = evt.particles[0]  # particle gun
    part.invpt = np.true_divide(part.q, part.pt)
    
    # Find the best sector
    sector_cnt_array = np.zeros((12,), dtype=np.int32)
    sector_hits_array = np.empty((12,), dtype=np.object)
    for ind in np.ndindex(sector_hits_array.shape):
      sector_hits_array[ind] = []
    
    for ihit, hit in enumerate(evt.hits):
      #print(".. hit  {0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11}".format(ihit, hit.bx, hit.type, hit.station, hit.ring, hit.sector, hit.fr, hit.sim_phi, hit.sim_theta, hit.time, hit.sim_tp1, hit.sim_tp2))
      
      assert(hit.emtf_phi < 5040)  # 84*60
      endsec = find_endsec(hit.endcap, hit.sector)
      sector_cnt_array[endsec] += 1
      sector_hits_array[endsec].append(hit)
    
    # Get the best sector hits
    best_sector = np.argmax(sector_cnt_array)
    sector_hits = sector_hits_array[best_sector]
    
    # The workhorse
    image_pixels, image_channels = make_emtf_image(sector_hits)
    out_image_pixels.append(image_pixels)
    out_image_channels.append(image_channels)
    
    labels = assign_emtf_label(part, best_sector)
    out_labels.append(labels)

  outs = (np.asarray(out_image_pixels, dtype=np.int32), 
          np.asarray(out_image_channels, dtype=np.float32),
          np.asarray(out_labels, dtype=np.int32),)
  return outs

In [77]:
image_pixels, image_channels, labels = process()
print image_pixels.shape, image_pixels
print image_channels.shape, image_channels
print labels.shape, labels

#outfile = 'histos_tbe.npz'
#np.savez_compressed(outfile, images=images, labels=labels)

[INFO] Opening file: ../test7/ntuple_SingleMuon_Toy_2GeV_add.6.root
Processing event: 0
(88, 50, 2) [[[ 19 223]
  [  6 221]
  [ 13 195]
  ...
  [-99 -99]
  [-99 -99]
  [-99 -99]]

 [[  7 163]
  [  8 162]
  [  5 166]
  ...
  [-99 -99]
  [-99 -99]
  [-99 -99]]

 [[107 203]
  [ 55 221]
  [ 52 203]
  ...
  [-99 -99]
  [-99 -99]
  [-99 -99]]

 ...

 [[ 31 178]
  [ 29 179]
  [ 26 192]
  ...
  [-99 -99]
  [-99 -99]
  [-99 -99]]

 [[ 15 108]
  [  4 106]
  [ 18 109]
  ...
  [-99 -99]
  [-99 -99]
  [-99 -99]]

 [[ 11  74]
  [  0  85]
  [ 13  88]
  ...
  [-99 -99]
  [-99 -99]
  [-99 -99]]]
(88, 50, 3) [[[-8.517465    0.20481928  0.        ]
  [-1.49701     0.16867469 10.        ]
  [-0.179778    0.22891566  7.        ]
  ...
  [        nan         nan         nan]
  [        nan         nan         nan]
  [        nan         nan         nan]]

 [[ 0.          0.03614458  0.        ]
  [-8.517465    0.06024097 12.        ]
  [ 0.149701    0.04819277  8.        ]
  ...
  [        nan         nan  

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (5,2.5)
#mpl.rcParams['axes.labelpad'] = 0
#mpl.rcParams['axes.labelsize'] = 0
#mpl.rcParams['xtick.labelsize'] = 0
#mpl.rcParams['ytick.labelsize'] = 0

In [ ]:
#aspect = float(strip_unit)/nlayers
#aspect = 20
aspect = 'auto'

image = images[0]
print image.shape
#print np.where(image[:,:,0])
#print np.where(image[:,:,1])
#print np.where(image[:,:,2])
#print np.where(image[:,:,3])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
#plt.imshow(image[:,:,1], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
#plt.imshow(image[:,:,2], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
#plt.imshow(image[:,:,3], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()

In [ ]:
image = images[1]
#print np.where(image[:,:,0])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()

In [ ]:
image = images[2]
#print np.where(image[:,:,0])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()

In [ ]:
image = images[3]
#print np.where(image[:,:,0])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()